## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\Jackeline\Desktop\Analysis_Projects\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nacajuca,MX,18.1333,-93.0167,82.45,78,49,4.61,scattered clouds
1,1,Bluff,NZ,-46.6000,168.3333,59.81,75,100,18.59,overcast clouds
2,2,Castro,BR,-24.7911,-50.0119,51.46,97,100,7.76,overcast clouds
3,3,Tasiilaq,GL,65.6145,-37.6368,48.36,85,18,7.83,few clouds
4,4,Aquiraz,BR,-3.9014,-38.3911,80.69,87,4,14.36,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_t = float(input("What is the minimum temperature (in °F) you would like for your holidays?"))
max_t = float(input("What is the maximum temperature (in °F) you would like for your holidays?"))

What is the minimum temperature (in °F) you would like for your holidays?70
What is the maximum temperature (in °F) you would like for your holidays?80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
criteria_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_t) & (city_data_df["Max Temp"] >= min_t)]

In [5]:
# 4a. Determine if there are any empty rows.
criteria_cities_df.count()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [6]:
# 4a. Determine if there are any empty rows.
criteria_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
criteria_cities_df.notnull().sum()

City_ID                195
City                   195
Country                195
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Iterate through the DataFrame.
clean_df = criteria_cities_df.dropna()
   

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Souillac,MU,71.98,broken clouds,-20.5167,57.5167,
6,Rikitea,PF,72.93,clear sky,-23.1203,-134.9692,
7,Namie,JP,76.71,overcast clouds,37.4833,141.0000,
22,Atuona,PF,77.72,few clouds,-9.8000,-139.0333,
31,Foundiougne,SN,77.77,broken clouds,14.1333,-16.4667,
43,Monrovia,LR,76.91,moderate rain,6.3005,-10.7969,
45,Chumphon,TH,73.51,overcast clouds,10.3333,98.8333,
54,Lancaster,US,73.65,clear sky,40.2504,-76.2499,
56,Uhlove,UA,71.56,overcast clouds,44.8215,33.6045,
65,Baffa,PK,78.85,overcast clouds,34.4378,73.2194,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found 
    #skip the city.
  
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output = "C:/Users/Jackeline/Desktop/Analysis_Projects/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))